<a href="https://colab.research.google.com/github/VivianOuou/NLP-Course/blob/main/course/en/chapter2/section3_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Models (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [11]:
!pip install datasets evaluate transformers[sentencepiece]

### Transformer 模型的不同加载方式详解

#### 1. 随机初始化模型（从零开始训练）
```python
from transformers import BertConfig, BertModel

# 创建默认配置
config = BertConfig()

# 初始化随机权重的模型
model = BertModel(config)
```
**特点**：
- 所有权重随机生成
- 输出无意义的噪声数据
- 需要从头训练（需大量数据和计算资源）
- 适用场景：
  - 学术研究新型架构
  - 特定领域全新预训练

> 📌 **注意**：完整的BERT预训练通常需要：
> - 16+块TPU/GPU
> - 数十GB文本数据
> - 数天到数周训练时间

#### 2. 加载预训练模型（推荐方式）
```python
from transformers import BertModel

# 加载Google官方预训练模型
model = BertModel.from_pretrained("bert-base-cased")
```
**优势**：
- 立即获得语言理解能力
- 支持快速微调（Fine-tuning）
- 节省大量计算资源

**模型标识符说明**：
- `bert-base-cased`：区分大小写的Base版
- `bert-large-uncased`：不区分大小写的Large版
- 更多变体可在[Hugging Face Model Hub](https://huggingface.co/models)查询

#### 3. 使用AutoModel的通用加载（最佳实践）
```python
from transformers import AutoModel

# 兼容所有BERT类模型的加载方式
model = AutoModel.from_pretrained("bert-base-cased")

# 同样适用于其他架构
roberta = AutoModel.from_pretrained("roberta-base")
distilbert = AutoModel.from_pretrained("distilbert-base-uncased")
```
**为什么推荐？**
- 代码可移植性强
- 无需修改代码即可切换模型架构
- 自动识别模型类型

#### 4. 模型缓存机制
首次加载时的自动行为：
1. 下载模型权重到缓存目录（默认`~/.cache/huggingface/transformers`）
2. 后续调用直接读取本地缓存

**自定义缓存位置**：
```bash
# 设置环境变量（Linux/macOS）
export HF_HOME=/path/to/your/cache

# Windows
set HF_HOME=C:\path\to\your\cache
```

#### 5. 模型加载过程图解
```mermaid
sequenceDiagram
    User->>HuggingFace: from_pretrained("bert-base-cased")
    HuggingFace->>Cache: 检查本地缓存
    alt 缓存存在
        Cache->>User: 直接加载
    else 缓存不存在
        HuggingFace->>Model Hub: 下载模型
        Model Hub->>HuggingFace: 返回权重文件
        HuggingFace->>Cache: 存储到本地
        Cache->>User: 加载模型
    end
```

#### 6. 不同加载方式对比
| 特性                | 随机初始化               | 预训练加载               | AutoModel加载         |
|--------------------|------------------------|------------------------|----------------------|
| **初始化方式**      | 随机权重                | 预训练权重              | 预训练权重           |
| **代码兼容性**      | 仅限指定架构            | 仅限指定架构            | 支持所有架构         |
| **训练需求**        | 需完整训练              | 可选择性微调            | 可选择性微调         |
| **典型应用**        | 新型模型研发            | 特定架构优化            | 快速实验/生产部署    |



In [12]:
from transformers import BertConfig, BertModel

# Building the config
config = BertConfig()

# Building the model from the config
model = BertModel(config)

In [13]:
print(config)

BertConfig {
  "_attn_implementation_autoset": true,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.50.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [18]:
from transformers import BertConfig, BertModel

config = BertConfig()
model = BertModel(config)


# Model is randomly initialized!

In [19]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

### Transformer 模型保存方法详解

#### 1. 基础保存方法
使用`save_pretrained()`方法可将模型保存到指定目录：
```python
model.save_pretrained("./my_bert_model/")
```
这将生成两个核心文件：
```
my_bert_model/
├── config.json        # 模型架构配置
└── pytorch_model.bin  # 模型权重参数
```

#### 2. 文件功能解析
| 文件名              | 内容类型 | 作用                                                                 |
|---------------------|----------|----------------------------------------------------------------------|
| `config.json`       | JSON文本 | 包含模型架构参数（如`hidden_size=768`）和训练元数据（框架版本等）     |
| `pytorch_model.bin` | 二进制   | 存储所有可训练参数的"状态字典"（state_dict）                         |

#### 3. 技术细节说明
- **PyTorch用户**：`pytorch_model.bin`是标准的PyTorch状态字典文件
- **TensorFlow用户**：保存时会额外生成`tf_model.h5`文件
- **多框架支持**：通过`save_pretrained(..., save_format="tf")`指定格式

#### 4. 完整保存示例
```python
from transformers import AutoModel

# 加载预训练模型
model = AutoModel.from_pretrained("bert-base-cased")

# 进行微调训练...
# model.train(...)

# 保存完整模型
model.save_pretrained(
    "./fine_tuned_bert/",
    save_config=True,  # 默认True
    save_function=torch.save,  # 可自定义保存方法
    state_dict=model.state_dict()  # 可传入自定义state_dict
)
```

#### 5. 文件内容示例
**config.json**:
```json
{
  "architectures": ["BertForMaskedLM"],
  "hidden_size": 768,
  "num_attention_heads": 12,
  "model_type": "bert",
  "transformers_version": "4.28.1"
}
```

**pytorch_model.bin**结构：
```python
{
  'embeddings.word_embeddings.weight': tensor(...),
  'encoder.layer.0.attention.self.query.weight': tensor(...),
  # ...其他数千个参数...
}
```

#### 6. 模型复用方法
保存后可通过两种方式重新加载：
```python
# 方式1：直接加载
from transformers import AutoModel
model = AutoModel.from_pretrained("./fine_tuned_bert/")

# 方式2：先加载配置再构建模型
from transformers import BertConfig, BertModel
config = BertConfig.from_pretrained("./fine_tuned_bert/config.json")
model = BertModel.from_pretrained("./fine_tuned_bert/", config=config)
```

#### 7. 高级保存选项
- **仅保存权重**：
  ```python
  torch.save(model.state_dict(), "weights_only.pt")
  ```
- **保存为不同格式：
  ```python
  model.save_pretrained("./tf_model/", save_format="tf")  # 保存TensorFlow格式
  ```

#### 8. 生产环境最佳实践
1. **版本控制**：将模型文件纳入Git LFS管理
2. **压缩分发**：使用`tar -czvf model.tar.gz ./model/`打包
3. **安全考虑**：
   ```python
   # 保存时排除敏感信息
   model.config.to_dict().pop("api_key", None)
   ```

#### 9. 常见问题解答
**Q：为什么需要同时保存config和weights？**
A：就像建筑需要"设计图+建筑材料"，config定义网络结构，weights提供具体参数值

**Q：如何保存到云存储？**
```python
# 示例：保存到Google Drive
model.save_pretrained("/content/drive/MyDrive/models/")
```

通过这种保存机制，您可以：
- 完整复现模型训练状态
- 方便地共享模型
- 灵活切换训练/推理环境

> 💡 提示：使用`push_to_hub()`方法可直接将模型上传到Hugging Face Hub：
> ```python
> model.push_to_hub("my-awesome-model")
> ```

In [20]:
model.save_pretrained("directory_on_my_computer")

Using a Transformer model for inference

Now that you know how to load and save a model, let’s try using it to make some predictions. Transformer models can only process numbers — numbers that the tokenizer generates. But before we discuss tokenizers, let’s explore what inputs the model accepts.

Tokenizers can take care of casting the inputs to the appropriate framework’s tensors, but to help you understand what’s going on, we’ll take a quick look at what must be done before sending the inputs to the model.

In [7]:
sequences = ["Hello!", "Cool.", "Nice!"]

In [8]:
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

In [9]:
import torch

model_inputs = torch.tensor(encoded_sequences)

In [10]:
output = model(model_inputs)